In [3]:
from dotenv import load_dotenv

load_dotenv()

True

### 1.準備

In [4]:
from langfuse import Langfuse

langfuse = Langfuse()

### 2. LangfuseからGenerations（生成AIの部分）のデータを取得

In [5]:
generations = langfuse.get_generations(limit=100)

In [11]:
from langchain.evaluation import Criteria, EvaluatorType, load_evaluator
from langchain_community.chat_models import BedrockChat
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

evaluator = load_evaluator(
    EvaluatorType.CRITERIA,
    criteria=Criteria.CONCISENESS.value,
    llm=ChatOpenAI(
        model="gpt-4o-mini",
        temperature=1,
    ),
)


In [12]:
generations

ObservationsViews(data=[ObservationsView(id='cdd4ab68-89e2-4011-9165-de4e785b0cdb', trace_id='509c5557-a118-47e3-ae5a-d7bd69eb7a27', type='GENERATION', name='ChatOpenAI', start_time=datetime.datetime(2024, 8, 17, 11, 36, 43, 851000, tzinfo=datetime.timezone.utc), end_time=datetime.datetime(2024, 8, 17, 11, 36, 46, 15000, tzinfo=datetime.timezone.utc), completion_start_time=None, model='gpt-4o-mini', model_parameters={'temperature': '0.0'}, input=[{'role': 'system', 'content': "You are an assistant for question-answering tasks.     Use the following pieces of retrieved context to answer the question.     If you don't know the answer, just say that you don't know.     Use three sentences maximum and keep the answer concise.\n    The advanced camera system on iPhone 15 and iPhone 15 Plus helps users capture everyday moments and cherished memories. A 48MP Main camera shoots sharp photos and videos while capturing fine details, with a quad-pixel sensor and 100 percent Focus Pixels for fast 

In [13]:
evaluator

CriteriaEvalChain(prompt=PromptTemplate(input_variables=['input', 'output'], partial_variables={'criteria': 'conciseness: Is the submission concise and to the point?'}, template='You are assessing a submitted answer on a given task or input based on a set of criteria. Here is the data:\n[BEGIN DATA]\n***\n[Input]: {input}\n***\n[Submission]: {output}\n***\n[Criteria]: {criteria}\n***\n[END DATA]\nDoes the submission meet the Criteria? First, write out in a step by step manner your reasoning about each criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then print only the single character "Y" or "N" (without quotes or punctuation) on its own line corresponding to the correct answer of whether the submission meets all criteria. At the end, repeat just the letter again by itself on a new line.'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x128181640>, async_client=<openai.resources.chat.complet

### 4.評価を実行する

In [14]:
eval_result = evaluator.evaluate_strings(
    input=generations.data[0].input, prediction=generations.data[0].output
)


In [15]:
eval_result

{'reasoning': "To determine if the submission meets the criterion of conciseness, I will evaluate the content step by step:\n\n1. **Definition of Conciseness**: Conciseness means providing necessary information in a clear and straightforward manner without superfluous details or excessive wording.\n\n2. **Analysis of Submission Length**: The submission contains three sentences. Each sentence delivers specific information about the iPhone 15's camera capabilities, such as megapixels, optical zoom levels, depth information in portrait photography, and improvements in night mode.\n\n3. **Relevance of Information**: The details provided are relevant to the user's request about the camera's performance. Each aspect mentioned adds value and pertains directly to the topic.\n\n4. **Avoidance of Redundancy**: There is no redundant language present. Each sentence communicates a distinct aspect of the camera's capabilities without unnecessary repetition.\n\n5. **Overall Clarity**: The submission 

### 5. 評価結果をLangfuseに登録する

In [16]:
langfuse.score(
    name=Criteria.CONCISENESS.value,
    trace_id=generations.data[0].trace_id,
    observation_id=generations.data[0].id,
    value=eval_result["score"],
    comment=eval_result["reasoning"],
)
